## Programmatic Access: Submit and Retrieve Artifacts using the SDMX Lab

This notebook demonstrates how to submit and retrieve artifacts using REST API calls to FMR running inside an SDMX Lab instance.

In [ ]:
from pathlib import Path
import requests
import certifi
from lxml import etree
from getpass import getpass
from requests.auth import HTTPBasicAuth

Set up your credentials for basic authorization. These will be used to authenticate REST API calls to the Lab instance.

In [3]:
user = input("Username: ")
password = getpass("Password: ")
auth = HTTPBasicAuth(username=user, password=password)

Specify the URL endpoint for the Lab instance. This is where the REST API calls will be sent.

In [4]:
# Define Lab instance endpoint
# This is constructed automatically from the provided user name.
fmr_url = f"https://{user}.sdmx.solutions/fmr"
endpoint = f"{fmr_url}/ws/secure/sdmxapi/rest"

### Submit a dataflow to FMR via REST-API

This step demonstrates how to submit a dataflow to FMR using the REST endpoint.

In [5]:
# Prepare SDMX resource
path = 'example_dataflow.xml'
dataflow = Path(path).read_text()
headers = {"Content-Type": "application/xml"}

response = requests.post(
    endpoint, 
    data=dataflow, 
    headers=headers, 
    auth=(user, password),
    verify=False
)

c:\Users\david.haslacher\AppData\Local\miniconda3\envs\sdmxlab\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'david.sdmx.solutions'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
# Display response
if response.status_code in [200, 201]:
    print("Resource successfully created in FMR.")
    print("Response:", response.text)
else:
    print(f"Failed to create resource. Status code: {response.status_code}")
    print("Response:", response.text)

Resource successfully created in FMR.
Response: <?xml version='1.0' encoding='UTF-8'?>
<message:RegistryInterface xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:message="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message" xmlns:com="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common" xmlns:reg="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/registry">
  <message:Header>
    <message:ID>IREF127969</message:ID>
    <message:Test>false</message:Test>
    <message:Prepared>2024-12-04T11:46:37Z</message:Prepared>
    <message:Sender id="Unknown"/>
  </message:Header>
  <message:SubmitStructureResponse/></message:RegistryInterface>


### Retrieve DSDs in submitted dataflow from FMR via REST-API

This step demonstrates how to retrieve the individual DSDs in the submitted dataflow from FMR using the REST endpoint.

In [7]:
endpoint_base = fmr_url + "/sdmx/v2/structure/datastructure"
root = etree.fromstring(dataflow.encode('utf-8'))

resource_urls = []
for dsd in root.findall(".//str:DataStructure", namespaces=root.nsmap):
    attributes = {key.lower(): value for key, value in dsd.attrib.items()}
    endpoint = f"{endpoint_base}/{attributes['agencyid'].strip()}/{attributes['id'].strip()}/{attributes['version'].strip()}"
    response = requests.get(endpoint, headers=headers, auth=auth, verify=False)
    print(f"Resource successfully retrieved. Status code: {response.status_code}.")
    print("Response:", response.text)

c:\Users\david.haslacher\AppData\Local\miniconda3\envs\sdmxlab\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'david.sdmx.solutions'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Resource successfully retrieved. Status code: 200.
Response: <?xml version='1.0' encoding='UTF-8'?><message:Structure xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:message="http://www.sdmx.org/resources/sdmxml/schemas/v3_0/message" xmlns:str="http://www.sdmx.org/resources/sdmxml/schemas/v3_0/structure" xmlns:com="http://www.sdmx.org/resources/sdmxml/schemas/v3_0/common"><message:Header><message:ID>IREF934485</message:ID><message:Test>false</message:Test><message:Prepared>2025-07-22T14:10:49Z</message:Prepared><message:Sender id="Unknown"/><message:Receiver id="not_supplied"/></message:Header><message:Structures><str:DataStructures><str:DataStructure urn="urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=BBK:BBK_BSBBK1(1.0)" isExternalReference="false" agencyID="BBK" id="BBK_BSBBK1" version="1.0"><com:Name xml:lang="de">Bankenstatistik, Bilanzstatistik, BISTA - Inlandsteil (MFI und Nicht-MFI-KI)</com:Name><str:DataStru

c:\Users\david.haslacher\AppData\Local\miniconda3\envs\sdmxlab\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'david.sdmx.solutions'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Resource successfully retrieved. Status code: 200.
Response: <?xml version='1.0' encoding='UTF-8'?><message:Structure xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xml="http://www.w3.org/XML/1998/namespace" xmlns:message="http://www.sdmx.org/resources/sdmxml/schemas/v3_0/message" xmlns:str="http://www.sdmx.org/resources/sdmxml/schemas/v3_0/structure" xmlns:com="http://www.sdmx.org/resources/sdmxml/schemas/v3_0/common"><message:Header><message:ID>IREF934486</message:ID><message:Test>false</message:Test><message:Prepared>2025-07-22T14:10:49Z</message:Prepared><message:Sender id="Unknown"/><message:Receiver id="not_supplied"/></message:Header><message:Structures><str:DataStructures><str:DataStructure urn="urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=HMS:DS_HMS_BISTA(1.0)" isExternalReference="false" agencyID="HMS" id="DS_HMS_BISTA" version="1.0"><com:Name xml:lang="en">Complete BISTA data</com:Name><str:DataStructureComponents><str:DimensionList urn="urn:sdmx:org.

c:\Users\david.haslacher\AppData\Local\miniconda3\envs\sdmxlab\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'david.sdmx.solutions'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
